<a href="https://colab.research.google.com/github/franklinscudder/DiffieHellmanExample/blob/main/DiffieHellman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip3 install numba --upgrade
!pip3 install pycryptodome

     |████████████████████████████████| 3.4MB 4.3MB/s 
     |████████████████████████████████| 25.3MB 1.6MB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


     |████████████████████████████████| 1.9MB 5.2MB/s 


In [16]:
from numba import njit
import random
from math import gcd as bltin_gcd
import Crypto.PublicKey.ECC as ecc

def primRoots(modulo):
    required_set = {num for num in range(1, modulo) if bltin_gcd(num, modulo) }
    return [g for g in range(1, modulo) if required_set == {pow(g, powers, modulo) for powers in range(1, modulo)}]

@njit
def Prime(n):
    if n & 1 == 0:
        return 2
    d= 3
    while d * d <= n:
        if n % d == 0:
            return d
        d= d + 2
    return 0

@njit
def getPrime(bits):
    p=1
    n = 0
    bits = 2 ** bits
    base = random.randint(bits/2, bits)

    while p is not 0:
        n+=1
        p = Prime(base + n)

    return base + n

class user:
    def __init__(self):
        self.rx = []
        self.secret = random.randint(1,4096)
    
    def initDH(self, target, bits):
        print(f"Finding a {bits}-bit prime...")
        self.p = getPrime(bits)
        print(f"Finding a suitable generator...")
        self.g = random.choice(primRoots(self.p))
        print(f"The shared prime is: {self.p}")
        print(f"The shared generator is: {self.g}")        
        self.myPubKey = (self.g ** self.secret) % self.p
        print(f"The initialiser's public key is: {self.myPubKey}")
        target.rx.append(self.p)
        target.rx.append(self.g)
        target.rx.append(self.myPubKey)
        target.respDH(self)
        self.theirPubKey = self.rx.pop()
        print(f"The respondant's public key is: {self.theirPubKey}")
        self.finalKey = (self.theirPubKey ** self.secret) % self.p
        print(f"The final agreed private key is: {self.finalKey}")

    def respDH(self, target):
        self.theirPubKey = self.rx.pop()
        self.g = self.rx.pop()
        self.p = self.rx.pop()
        self.myPubKey = (self.g ** self.secret) % self.p
        target.rx.append(self.myPubKey)
        self.finalKey = (self.theirPubKey ** self.secret) % self.p

    def initECDHE(self, target):
        print("Using NIST P-256 elliptic curve...")
        print("Generating a new initialiser key pair...")
        self.privEC = ecc.generate(curve="p256")
        self.pubEC = self.privEC.public_key()
        print("Sending initialiser's public key...")
        target.rx.append(self.pubEC)
        target.respECDHE(self)
        self.theirPubEC = self.rx.pop()
        print("Recieved respondant's key!")
        self.finalEC = (self.privEC.d * self.theirPubEC.pointQ)
        self.finalECx = self.finalEC.x
        print(f"Final shared secret is: {self.finalECx}")
        
    def respECDHE(self, target):
        print("Generating a new respondant key pair...")
        self.privEC = ecc.generate(curve="p256")
        self.pubEC = self.privEC.public_key()
        self.theirPubEC = self.rx.pop()
        print("Recieved initialiser's key!")
        print("Sending respondant's public key...")
        target.rx.append(self.pubEC)
        self.finalEC = self.privEC.d * self.theirPubEC.pointQ
        self.finalECx = self.finalEC.x
        print(f"Final shared secret is: {self.finalECx}")
        

if __name__ == "__main__":
    alice = user()
    bob = user()

    print("+++ Modulo Exponential Diffie-Hellman +++")
    alice.initDH(bob, 4)
    print()

    print("+++ Elliptic Curve Diffie-Hellman (Ephemeral) +++")
    bob.initECDHE(alice)


+++ Modulo Exponential Diffie-Hellman +++
Finding a 4-bit prime...
Finding a suitable generator...
The shared prime is: 11
The shared generator is: 6
The initialiser's public key is: 6
The respondant's public key is: 7
The final agreed private key is: 7

+++ Elliptic Curve Diffie-Hellman (Ephemeral) +++
Using NIST P-256 elliptic curve...
Generating a new initialiser key pair...
Sending initialiser's public key...
Generating a new respondant key pair...
Recieved initialiser's key!
Sending respondant's public key...
Final shared secret is: 11474164584863126621860694098576189739003695545082394056251958880321190646056
Recieved respondant's key!
Final shared secret is: 11474164584863126621860694098576189739003695545082394056251958880321190646056
